In [1]:
%load_ext nb_black

import pandas as pd
import sqlite3 as sql
import numpy as np
import chardet

<IPython.core.display.Javascript object>

In [2]:
first_5 = pd.read_csv("crunchbase-investments.csv", nrows=5)
first_5

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


<IPython.core.display.Javascript object>

In [3]:
first_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   company_permalink       5 non-null      object
 1   company_name            5 non-null      object
 2   company_category_code   5 non-null      object
 3   company_country_code    5 non-null      object
 4   company_state_code      4 non-null      object
 5   company_region          5 non-null      object
 6   company_city            4 non-null      object
 7   investor_permalink      5 non-null      object
 8   investor_name           5 non-null      object
 9   investor_category_code  4 non-null      object
 10  investor_country_code   5 non-null      object
 11  investor_state_code     5 non-null      object
 12  investor_region         5 non-null      object
 13  investor_city           5 non-null      object
 14  funding_round_type      5 non-null      object
 15  funded_at 

<IPython.core.display.Javascript object>

#### Observations:
- A quick glance at the first 5 rows shows us there are columns, 18 of which are object type.
- `year` and `raised_amount_used` are appropriately int64 type for these rows.
- Both `permalink` rows can be ignored as they will not add to the analysis.
- Additionally, we can ignore the redundant `funded_month`, `funded_quarter`, and `funded_year` columns, if `funded_at` is not missing more values than these columns.
- `funded_at` can be parsed as datetime type.
- The remainder of the columns appear appropriate for typecasting as category, but we will verify their unique values.

## Batch Processing Preparation

#### Creating List of Columns to Include

In [4]:
usecols = first_5.columns.tolist()
usecols.remove("company_permalink")
usecols.remove("investor_permalink")
print(f"We are currently including {len(usecols)} columns.")

We are currently including 18 columns.


<IPython.core.display.Javascript object>

#### Assigning Variables for `pd.read_csv()`

In [5]:
file = "crunchbase-investments.csv"
chunksize = 5000
encoding = "latin1"

<IPython.core.display.Javascript object>

#### Calculating Total Number of Rows

In [6]:
chunk_iter = pd.read_csv(file, chunksize=chunksize, encoding=encoding, usecols=usecols)
total_rows = np.array([len(chunk) for chunk in chunk_iter]).sum()
print(f"There are {total_rows} total rows in the dataset.")

There are 52870 total rows in the dataset.


<IPython.core.display.Javascript object>

#### Checking Memory Footprint by Chunk for `chunksize=5000`

In [7]:
print("Memory for Each Chunk in MB:")
chunk_iter = pd.read_csv(file, chunksize=chunksize, encoding=encoding, usecols=usecols)
[chunk.memory_usage(deep=True).sum() / (2**20) for chunk in chunk_iter]

Memory for Each Chunk in MB:


[4.807390213012695,
 4.689476013183594,
 4.693775177001953,
 4.691075325012207,
 4.679686546325684,
 4.702727317810059,
 4.686784744262695,
 4.677568435668945,
 4.583010673522949,
 3.912022590637207,
 2.2458419799804688]

<IPython.core.display.Javascript object>

#### Observations:
- We are staying comfortably below the 50% mark of our allotted RAM of 10 MB with `chunksize=5000`.

#### Calculating Initial Memory Footprint in MB by Column and Total

In [8]:
print("Initial Memory Footprint in MB:\n")
chunk_iter = pd.read_csv(
    file,
    chunksize=chunksize,
    usecols=usecols,
    encoding=encoding,
)
chunk_memory = pd.concat([chunk.memory_usage(deep=True) for chunk in chunk_iter])
initial_footprint = chunk_memory.groupby(chunk_memory.index).sum() / (2**20)
print(initial_footprint, "\n")
print("Total: ", np.round(initial_footprint.sum(), 2), "MB")

Initial Memory Footprint in MB:

Index                     0.001381
company_category_code     3.262619
company_city              3.343512
company_country_code      3.025223
company_name              3.424955
company_region            3.253541
company_state_code        2.962161
funded_at                 3.378091
funded_month              3.226837
funded_quarter            3.226837
funded_year               0.403366
funding_round_type        3.252704
investor_category_code    0.593590
investor_city             2.751430
investor_country_code     2.524654
investor_name             3.734270
investor_region           3.238946
investor_state_code       2.361876
raised_amount_usd         0.403366
dtype: float64 

Total:  48.37 MB


<IPython.core.display.Javascript object>

#### Observations:
- With the default column datatypes, the datasets's initial memory footprint is ~48 MB.

#### Verifying Default Datatypes by Chunk

In [9]:
chunk_iter = pd.read_csv(file, chunksize=chunksize, encoding=encoding, usecols=usecols)
chunk_dtypes = pd.concat([chunk.dtypes for chunk in chunk_iter])
overall_dtypes = chunk_dtypes.groupby(chunk_dtypes.index).value_counts()
overall_dtypes

company_category_code   object     11
company_city            object     11
company_country_code    object     11
company_name            object     11
company_region          object     11
company_state_code      object     11
funded_at               object     11
funded_month            object     11
funded_quarter          object     11
funded_year             int64      10
                        float64     1
funding_round_type      object     11
investor_category_code  float64    10
                        object      1
investor_city           object      9
                        float64     2
investor_country_code   object      9
                        float64     2
investor_name           object     11
investor_region         object     11
investor_state_code     object      9
                        float64     2
raised_amount_usd       float64    11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have 5 columns that change datatype across chunks.
- `funded_year` has some float values, that are likely NaNs, but is consistently numeric.
- `investor_category_code` has one chunk that is object rather than float.
- `investor_city`, `investor_country_code`, and `investor_state_code` have 2 chunks that are float rather than object.
- Let us check the missing values before making any decisions.

#### Checking Missing Values by Column

In [10]:
print("Percentage of Missing Values:")
chunk_iter = pd.read_csv(file, chunksize=chunksize, encoding=encoding, usecols=usecols)
chunk_missing = pd.concat([chunk.isna().sum() for chunk in chunk_iter])
total_perc_missing = chunk_missing.groupby(chunk_missing.index).sum() / total_rows * 100
total_perc_missing

Percentage of Missing Values:


company_category_code      1.216191
company_city               1.008133
company_country_code       0.001891
company_name               0.001891
company_region             0.001891
company_state_code         0.930584
funded_at                  0.005674
funded_month               0.005674
funded_quarter             0.005674
funded_year                0.005674
funding_round_type         0.005674
investor_category_code    95.379232
investor_city             23.605069
investor_country_code     22.699073
investor_name              0.003783
investor_region            0.003783
investor_state_code       31.793077
raised_amount_usd          6.807263
dtype: float64

<IPython.core.display.Javascript object>

#### Observations:
- `investor_category_code` almost all missing values, so we will ignore it.
- `investor_city`, `investor_country_code`, and `investor_state_code` all have a high proportion of missing values.  In practice, we would discuss the analysis requirements with the domain expert or assess the key questions of the analysis before making a decision.  For the purpose of this example project, we will ignore these columns, assuming some of the same information is captured by `investor_region` that has very few null values.
- The same applies to the columns containing temporal information.  All of the `funded_` columns have the same (very low) number of missing values, so they are likely the same rows.  In practice that would be confirmed.  Here, we will keep `funded_at` to typecast as datetime and `funding_round_type` and ignore the others.

#### Updating `usecols`

In [11]:
cols_to_ignore = [
    "investor_category_code",
    "investor_city",
    "investor_country_code",
    "investor_state_code",
    "funded_month",
    "funded_quarter",
    "funded_year",
]
for col in cols_to_ignore:
    usecols.remove(col)
print(f"There are {len(usecols)} remaining columns in usecols.")

There are 11 remaining columns in usecols.


<IPython.core.display.Javascript object>

## Optimizing Data Types
- We are ignoring all of the columns that had variation in datatype across chunks, so we are left with 11 object type columns and 1 float type to optimize.
- Let us first instantiate a dictionary for `dtypes` and assign some parameters of `pd.read_csv()` to collect our decisions as we proceed.

#### Instantiating `dtypes`, `converters`, and  `date_parser` parameters and Creating Lists for Numeric and Object Columns

In [14]:
dtypes = {}
converters = {
    "funded_at": lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors="coerce")
}

num_cols = ["raised_amount_usd"]
obj_cols = usecols.copy()
obj_cols.remove("raised_amount_usd")
obj_cols.remove("funded_at")

<IPython.core.display.Javascript object>

#### Checking Unique Value Counts for Object Type Columns

In [15]:
for col in obj_cols:
    chunk_iter = pd.read_csv(
        file,
        chunksize=chunksize,
        encoding=encoding,
        usecols=usecols,
        converters=converters,
    )
    chunk_vc = pd.concat([chunk[col].value_counts() for chunk in chunk_iter])
    total_vc = chunk_vc.groupby(chunk_vc.index).sum().sort_values(ascending=False)
    print(col, "\n")
    print(total_vc, "-" * 50)

company_name 

ecomom                            58
Fab.com                           57
Aperto Networks                   49
Practice Fusion                   47
Klout                             46
                                  ..
ScribeStorm                        1
FindMySong                         1
Find That File                     1
Financial Transaction Services     1
Edupath                            1
Name: company_name, Length: 11573, dtype: int64 --------------------------------------------------
company_category_code 

software            7243
web                 5015
biotech             4951
enterprise          4489
mobile              4067
advertising         3200
ecommerce           2168
cleantech           1948
games_video         1893
analytics           1863
hardware            1537
medical             1315
semiconductor       1292
network_hosting     1075
security             996
finance              931
social               920
education            783
healt

<IPython.core.display.Javascript object>

#### Observations:
- There is at least one problematic row, for entries are shifted for `company_category_code`, `company_country_code`, `company_state_code`, and `company_region`.

#### Checking for Erroneous Rows

In [16]:
chunk_iter = pd.read_csv(
    file,
    chunksize=chunksize,
    encoding=encoding,
    usecols=usecols,
    converters=converters,
)
for chunk in chunk_iter:
    index = (
        (chunk["company_category_code"] == "2/7/08")
        | (chunk["company_country_code"] == "2008-02")
        | (chunk["company_state_code"] == "2008-Q1")
        | (chunk["company_region"] == "2008")
    )
    if len(chunk.loc[index, :]) > 0:
        print(chunk.loc[index, :])

      company_name company_category_code company_country_code  \
34226    series-c+                2/7/08              2008-02   

      company_state_code company_region company_city investor_name  \
34226            2008-Q1           2008     10000000           NaN   

      investor_region funding_round_type funded_at  raised_amount_usd  
34226             NaN                NaN       NaT                NaN  


<IPython.core.display.Javascript object>

#### Observations:
- The entry at index 34226 has information in the wrong columns.
- This row could be skipped using the `skiprows` parameter of `pd.read_csv()`.
- For our purposes here we will include it.

#### Checking Total Number of Unique Values for Object Columns

In [17]:
for col in obj_cols:
    chunk_iter = pd.read_csv(
        file,
        chunksize=chunksize,
        encoding=encoding,
        usecols=[col],
        converters=converters,
    )
    chunk_vc = pd.concat([chunk[col].value_counts() for chunk in chunk_iter])
    num_unique = len(set(chunk_vc.index))
    print(col, ": ", num_unique)

company_name :  11573
company_category_code :  43
company_country_code :  2
company_state_code :  50
company_region :  546
company_city :  1229
investor_name :  10465
investor_region :  585
funding_round_type :  9


<IPython.core.display.Javascript object>

#### Observations:
- The following columns can be read into Python as category type:
    - `company_category_code`
    - `company_country_code`
    - `company_state_code`
    - `funding_round_type`
- There is some variation in case of the first letter in `company_city` that may occur elsewhere.

#### Updating `dtypes` for Category Columns

In [18]:
cat_cols = [
    "company_category_code",
    "company_country_code",
    "company_state_code",
    "funding_round_type",
]
for col in cat_cols:
    dtypes[col] = "category"

<IPython.core.display.Javascript object>

#### Checking Float Column for Potential Downcasting to Smaller Subtype

In [19]:
mapping = {"float16": 0, "float32": 1, "float64": 2}
for col in num_cols:
    low_type = "float16"
    chunk_iter = pd.read_csv(
        file,
        chunksize=chunksize,
        usecols=[col],
        encoding=encoding,
        converters=converters,
    )
    chunk[col] = pd.to_numeric(chunk[col], downcast="float")
    if mapping[str(chunk[col].dtype)] > mapping[str(low_type)]:
        low_type = chunk[col].dtype
    dtypes[col] = str(low_type)

dtypes

{'company_category_code': 'category',
 'company_country_code': 'category',
 'company_state_code': 'category',
 'funding_round_type': 'category',
 'raised_amount_usd': 'float32'}

<IPython.core.display.Javascript object>

#### Observations:
- We were able to find more optimal datatypes for 6 of the 12 columns that we are including, including parsing the date for `funded_at`.
- Though some of the remaining object type columns have under 50% unique values, their potential for variability lends itself to remaining as object type, for now.

## Checking the Anticipated Memory Footprint

In [20]:
print("Anticipated Memory Footprint for Batch Processing in MB:\n")
chunk_iter = pd.read_csv(
    file,
    chunksize=chunksize,
    encoding=encoding,
    usecols=usecols,
    converters=converters,
    dtype=dtypes,
)
chunk_memory = pd.concat([chunk.memory_usage(deep=True) for chunk in chunk_iter])
final_footprint = chunk_memory.groupby(chunk_memory.index).sum() / (2**20)
print(final_footprint)
print("Total: ", final_footprint.sum(), "MB")

Anticipated Memory Footprint for Batch Processing in MB:

Index                    0.001381
company_category_code    0.091980
company_city             3.343512
company_country_code     0.051188
company_name             3.424955
company_region           3.253541
company_state_code       0.091649
funded_at                0.403366
funding_round_type       0.059248
investor_name            3.734270
investor_region          3.238946
raised_amount_usd        0.201683
dtype: float64
Total:  17.895718574523926 MB


<IPython.core.display.Javascript object>

#### Observations:
- We just received a request to get the entire memory footprint below 10 MB.
- To do so, we will proceed to cast `company_region` and `investor_region` as category, then recheck the memory footprint.

#### Adding More Object Columns to `dtypes`

In [24]:
for col in ["company_region", "investor_region"]:
    dtypes[col] = "category"

<IPython.core.display.Javascript object>

#### Re-checking Memory Footprint

In [25]:
print("Anticipated Memory Footprint for Batch Processing in MB:\n")
chunk_iter = pd.read_csv(
    file,
    chunksize=chunksize,
    encoding=encoding,
    usecols=usecols,
    converters=converters,
    dtype=dtypes,
)
chunk_memory = pd.concat([chunk.memory_usage(deep=True) for chunk in chunk_iter])
final_footprint = chunk_memory.groupby(chunk_memory.index).sum() / (2**20)
print(final_footprint)
print("Total: ", final_footprint.sum(), "MB")

Anticipated Memory Footprint for Batch Processing in MB:

Index                    0.001381
company_category_code    0.091980
company_city             0.624051
company_country_code     0.051188
company_name             3.424955
company_region           0.317376
company_state_code       0.091649
funded_at                0.403366
funding_round_type       0.059248
investor_name            1.163946
investor_region          0.217028
raised_amount_usd        0.201683
dtype: float64
Total:  6.64785099029541 MB


<IPython.core.display.Javascript object>

#### Observations:
- That got us under 10 MB.
- We can no proceed to loading the data into a sqlite database.
- We have been asked to load each chunk as a new table in the database.

## Loading Chunks into SQLite

In [28]:
conn = sql.connect("crunchbase.db")
chunk_iter = pd.read_csv(
    file,
    chunksize=chunksize,
    encoding=encoding,
    converters=converters,
    usecols=usecols,
    dtype=dtypes,
)
for i, chunk in enumerate(chunk_iter):
    chunk.to_sql(f"investments_{i}", conn, if_exists="append", index=False)

<IPython.core.display.Javascript object>

## Test Query

In [44]:
df = pd.read_sql("SELECT * FROM investments_10 LIMIT 5;", conn, parse_dates="funded_at")
df

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_region,funding_round_type,funded_at,raised_amount_usd
0,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,unknown,series-a,2012-10-01,3060000.0
1,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,unknown,series-b,2007-10-01,12000000.0
2,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,unknown,angel,2008-04-18,500000.0
3,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,unknown,angel,2010-01-01,750000.0
4,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,unknown,angel,2010-01-01,NaN


<IPython.core.display.Javascript object>

In [45]:
df.dtypes

company_name                     object
company_category_code            object
company_country_code             object
company_state_code               object
company_region                   object
company_city                     object
investor_name                    object
investor_region                  object
funding_round_type               object
funded_at                datetime64[ns]
raised_amount_usd               float64
dtype: object

<IPython.core.display.Javascript object>